In [23]:
from __future__ import print_function
from __future__ import absolute_import
from distutils.dir_util import copy_tree

import os
import sys
import glob
import json
import re
import shutil
from shutil import copytree, ignore_patterns
import numpy as np

CONTEXT_LENGTH = 48
IMAGE_SIZE = 256
BATCH_SIZE = 64
EPOCHS = 10
STEPS_PER_EPOCH = 72000

class Utils:
    @staticmethod
    def sparsify(label_vector, output_size):
        sparse_vector = []

        for label in label_vector:
            sparse_label = np.zeros(output_size)
            sparse_label[label] = 1

            sparse_vector.append(sparse_label)

        return np.array(sparse_vector)

    @staticmethod
    def get_preprocessed_img(img_path, image_size):
        import cv2
        img = cv2.imread(img_path)
        img = cv2.resize(img, (image_size, image_size))
        img = img.astype('float32')
        img /= 255
        return img

    @staticmethod
    def show(image):
        import cv2
        cv2.namedWindow("view", cv2.WINDOW_AUTOSIZE)
        cv2.imshow("view", image)
        cv2.waitKey(0)
        cv2.destroyWindow("view")

print("Converting images to numpy arrays...")
input_path = "img_align_celeba"
output_path = "img_align_celeba"
for f in os.listdir(input_path):
    if f.find(".jpg") != -1:
        img = Utils.get_preprocessed_img("{}/{}".format(input_path, f), IMAGE_SIZE)
        file_name = f[:f.find(".jpg")]

        np.savez_compressed("{}/{}".format(output_path, file_name), features=img)
        retrieve = np.load("{}/{}.npz".format(output_path, file_name))["features"]

        assert np.array_equal(img, retrieve)


print("Numpy arrays saved in {}".format(output_path))

Converting images to numpy arrays...
Numpy arrays saved in img_align_celeba


In [25]:
arrays = np.load("all_images.npz")
arrays

In [11]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.utils import to_categorical

In [3]:



print('Training data shape : ', train_x.shape, train_y.shape)
print('Testing data shape : ', test_x.shape, test_y.shape)

# Find the unique numbers from the train labels
classes = np.unique(train_y)
classes_num = len(classes)
print('Total number of outputs : ', classes_num)
print('Output classes : ', classes)

plt.figure(figsize=[10,5])

# Display the first image in training data
plt.subplot(121)
plt.imshow(train_x[0,:,:], cmap='copper')
plt.title("Ground Truth : {}".format(train_y[0]))
plt.axis("off")

# Display the first image in testing data
plt.subplot(122)
plt.imshow(test_x[0,:,:], cmap='copper')
plt.title("Ground Truth : {}".format(test_y[0]))
plt.axis("off")

ValueError: too many values to unpack (expected 2)

#### 2. Preprocessing

In [6]:
# Change from matrix to array of dimension 28x28 to array of dimension 784

train_x = train_x.reshape(train_x.shape[0], -1)
test_x = test_x.reshape(test_x.shape[0], -1)

# 0-255 to 0-1
train_x = train_x/255
test_x = test_x/255

# Change the labels from integer to categorical data
train_y_one_hot = keras.utils.to_categorical(train_y)
test_y_one_hot = keras.utils.to_categorical(test_y)

In [7]:
train_x.shape

(60000, 784)

#### 3. Model Configuration

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(train_x.shape[1],)))
model.add(Dense(512, activation='relu'))
model.add(Dense(classes_num, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_x, train_y_one_hot,
                   batch_size=256, epochs=10, verbose=True,
                   validation_data=(test_x, test_y_one_hot))

[test_loss, test_acc] = model.evaluate(test_x, test_y_one_hot)
print(f"Evaluation result on Test Data : Loss = {test_loss}, accuracy = {test_acc}")



Epoch 1/10
235/235 [==============================] - 5s 18ms/step - loss: 0.2723 - accuracy: 0.9160 - val_loss: 0.1181 - val_accuracy: 0.9634
Epoch 2/10
235/235 [==============================] - 4s 16ms/step - loss: 0.0949 - accuracy: 0.9706 - val_loss: 0.1159 - val_accuracy: 0.9620
Epoch 3/10
235/235 [==============================] - 4s 15ms/step - loss: 0.0576 - accuracy: 0.9815 - val_loss: 0.1020 - val_accuracy: 0.9679
Epoch 4/10
235/235 [==============================] - 4s 15ms/step - loss: 0.0395 - accuracy: 0.9872 - val_loss: 0.0701 - val_accuracy: 0.9799
Epoch 5/10
235/235 [==============================] - 4s 19ms/step - loss: 0.0289 - accuracy: 0.9911 - val_loss: 0.0763 - val_accuracy: 0.9786
Epoch 6/10
235/235 [==============================] - 4s 16ms/step - loss: 0.0209 - accuracy: 0.9931 - val_loss: 0.0661 - val_accuracy: 0.9835
Epoch 7/10
235/235 [==============================] - 4s 18ms/step - loss: 0.0165 - accuracy: 0.9945 - val_loss: 0.0946 - val_accuracy: 0.9752